In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

from ib_insync import util, LimitOrder, IB
util.startLoop()

from loguru import logger

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>
from src.utils import (Vars, get_pickle, 
                       quick_pf, place_orders)

_vars = Vars(MARKET)
PORT = port = _vars.PORT

DATAPATH = ROOT / 'data' / MARKET

In [ ]:
df_tgt = get_pickle(DATAPATH / "df_naked_targets.pkl")
df = df_tgt.dropna(subset=['expPrice'])


In [ ]:
# REMOVE PORTFOLIO SYMBOLS FROM TARGET
# ====================================

# Get the portfolio
with IB().connect(port=PORT) as ib:
    df_pf = quick_pf(ib)

# Remove targets which are already in the portfolio
if df_pf:
    df = df[~df.symbol.isin(set(df_pf.symbol))]

In [ ]:
# REMOVE OPEN ORDER SYMBOLS
# =========================

with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = ["conId", "symbol", "secType", "expiry",
        "strike", "right", "orderId", "permId", "action",
        "totalQuantity", "lmtPrice", "status",]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]

else:
    df_openords = None
    

try:
    if df_openords:  # There are some open orders
        df = df[~df.symbol.isin(set(df_openords.symbol))]
except ValueError:
    pass

In [ ]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]